In [1]:
import ast
import os

import sys
import tqdm
import pandas as pd

from datasets import Dataset
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from pathlib import Path

from ragas import EvaluationDataset
from ragas import evaluate
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from ragas.run_config import RunConfig

sys.path.insert(0, str(Path().resolve().parent))

from app.services.rag_pipeline.model import HybridRetriever, AnswerGenerator

In [3]:
testdataset = pd.read_csv('~/Work/data/q_insigt/q-insigt-testset.csv', index_col=False)

In [4]:
testdataset.head()

,user_input,reference_contexts,reference,synthesizer_name
0,Что произошло с Габеном в последнее время?,"['POV – казуальный Габен, когда все ждут геймп...","Недавно Габен стал казуальным, когда все ждали...",single_hop_specifc_query_synthesizer
1,Что такое Kafka и почему это важно для работы ...,['#вакансия #Middle #DataAnalyst #fulltime #уд...,"Kafka — это система, которая используется для ...",single_hop_specifc_query_synthesizer
2,Что такое Висп и почему его упоминают вместе с...,['Снова что-то на квинновском 🆗 Легко говорить...,Висп упоминается в контексте Лешего как одна и...,single_hop_specifc_query_synthesizer
3,Какой контракт выиграла Lockheed Martin и на к...,['Lockheed Martin выиграла контракт на противо...,Lockheed Martin выиграла контракт на противора...,single_hop_specifc_query_synthesizer
4,Кто получает мерч?,"['Привет, Чемпионы!🏆\n\nСегодня произошло знам...","Мерч получает каждый, кто получил сертификат и...",single_hop_specifc_query_synthesizer


In [5]:
testdataset['text'] = testdataset.reference_contexts.apply(lambda x: ast.literal_eval(x)[0])

In [6]:
data_path = "~/Work/data/q_insigt/telegram_posts.csv"
df = pd.read_csv(data_path)
df = df.dropna(subset=['text'])

In [7]:
df.head()

,date,name,text,views,comments,forwards,emoji,reactions,subscribers
0,2024-06-27 14:59:34,🏄 Соревновательный Data Science | Kaggle | Чем...,"**👨‍💻**** Апдейт про Kaggle Camp, который вы ж...",755.0,2,3,"👍: 14, ⚡: 7, ❤‍🔥: 3",24,2672
1,2024-06-25 15:33:36,🏄 Соревновательный Data Science | Kaggle | Чем...,👨‍💻 **Контент из сообщества**! \n**\n****🏄****...,877.0,3,19,"👍: 16, Custom emoji: 7",23,2672
2,2024-06-24 12:06:49,🏄 Соревновательный Data Science | Kaggle | Чем...,🏆 [**Запись стрима**](https://youtu.be/kz__54Y...,1075.0,0,11,"❤‍🔥: 10, Custom emoji: 2",12,2672
3,2024-06-21 14:10:38,🏄 Соревновательный Data Science | Kaggle | Чем...,Стрим случился! Всех причастных рады были увид...,1348.0,9,2,❤‍🔥: 4,4,2672
4,2024-06-21 12:45:52,🏄 Соревновательный Data Science | Kaggle | Чем...,"Привет, работяги! 👨‍💻\nКак неделя прошла? Есть...",1348.0,4,2,❤‍🔥: 10,10,2672


In [8]:
testdataset = testdataset.merge(df[['text', 'name']], on='text')

In [9]:
testdataset.head()

,user_input,reference_contexts,reference,synthesizer_name,text,name
0,Что произошло с Габеном в последнее время?,"['POV – казуальный Габен, когда все ждут геймп...","Недавно Габен стал казуальным, когда все ждали...",single_hop_specifc_query_synthesizer,"POV – казуальный Габен, когда все ждут геймпле...",BetBoom Esports Dota 2
1,Что такое Kafka и почему это важно для работы ...,['#вакансия #Middle #DataAnalyst #fulltime #уд...,"Kafka — это система, которая используется для ...",single_hop_specifc_query_synthesizer,#вакансия #Middle #DataAnalyst #fulltime #удал...,Data Science Jobs
2,Что такое Висп и почему его упоминают вместе с...,['Снова что-то на квинновском 🆗 Легко говорить...,Висп упоминается в контексте Лешего как одна и...,single_hop_specifc_query_synthesizer,"Снова что-то на квинновском 🆗 Легко говорить, ...",BetBoom Esports Dota 2
3,Какой контракт выиграла Lockheed Martin и на к...,['Lockheed Martin выиграла контракт на противо...,Lockheed Martin выиграла контракт на противора...,single_hop_specifc_query_synthesizer,Lockheed Martin выиграла контракт на противора...,"Больше, чем экономика"
4,Кто получает мерч?,"['Привет, Чемпионы!🏆\n\nСегодня произошло знам...","Мерч получает каждый, кто получил сертификат и...",single_hop_specifc_query_synthesizer,"Привет, Чемпионы!🏆\n\nСегодня произошло знамен...",🏄 Соревновательный Data Science | Kaggle | Чем...


In [13]:
embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-base-v2")
retriever = HybridRetriever(
    uri="http://localhost:54637",
    collection_name="main_e5",
    dense_embedding_function=embeddings
)
retriever.build_collection(recreation=False)

Коллекция уже есть и не пересоздаётся


'Collection already exists'

In [12]:
embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-base-v2")
retriever = HybridRetriever(
    uri="http://localhost:54637",
    collection_name="main_e5",
    dense_embedding_function=embeddings
)
retriever.build_collection(recreation=True)

for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0]):
    retriever.insert_data(
        {"message": row['text'], 'chat_name': row['name'], 'chat_message_id': index}
    )

Создали новую коллекцию  в Milvus


100%|███████████████████████████████████████| 4508/4508 [06:33<00:00, 11.45it/s]


In [14]:
generator = AnswerGenerator(retriever)
retriever_params = {
    'k': 20,
    'mode': 'hybrid',
    'k_rerank': 10,
}

dataset = []
for index, row in tqdm.tqdm(testdataset.iterrows(), total=testdataset.shape[0]):
    answer, context = await generator.generate_answer(
        row['user_input'],
        chat_names=[row['name']],
        retriever_params=retriever_params
    )

    dataset.append(
        {
            "user_input": row['user_input'],
            "retrieved_contexts": context,
            "response": answer,
            "reference": row['reference']
        }
    )

100%|█████████████████████████████████████████| 100/100 [05:23<00:00,  3.24s/it]


In [15]:
eval_dataset = EvaluationDataset.from_list(dataset)

In [16]:
llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

In [17]:
metrics=[
    LLMContextRecall(), 
    Faithfulness(), 
    FactualCorrectness(),
]

In [18]:
evaluator_llm = LangchainLLMWrapper(llm)

result = evaluate(
    dataset=eval_dataset, 
    llm=llm,
    embeddings=embeddings,
    metrics=metrics,
    run_config=RunConfig(
        timeout=180,
        max_retries=10,
        max_wait = 180,
        max_workers= 1, 
    ),
)

Evaluating:   0%|          | 0/300 [00:00<?, ?it/s]

Exception raised in Job[14]: TimeoutError()
Exception raised in Job[67]: TimeoutError()
Exception raised in Job[190]: TimeoutError()
Exception raised in Job[213]: TimeoutError()
Exception raised in Job[241]: TimeoutError()
Exception raised in Job[244]: TimeoutError()
Exception raised in Job[283]: APIConnectionError(Connection error.)
Exception raised in Job[289]: APIConnectionError(Connection error.)


In [19]:
result

{'context_recall': 0.9512, 'faithfulness': 0.9178, 'factual_correctness': 0.6451}

### Результаты валидации

| Embedding model  | Ranker                              | Context Recall | Faithfulness | Factual Correctness |
|------------------|-------------------------------------|----------------|--------------|---------------------|
| all-MiniLM-L6-v2 | Weighted (k=20, weights=[0.5, 0.5]) | 0.9456         | 0.9011       | 0.6360              |
| all-MiniLM-L6-v2 | RRF (k=20, k_rerank=10)             | 0.9400         | 0.9173       | 0.6091              |
| e5-base-v2       | Weighted (k=20, weights=[0.5, 0.5]) | 0.9596         | 0.9200       | 0.6236              |
| e5-base-v2       | RRF (k=20, k_rerank=10)             | 0.9512         | 0.9178       | 0.6451              |